In [8]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx
import ast
import pathlib
import time
import multiprocessing as mp
import itertools
%matplotlib inline

# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

ox.__version__

'1.0.1'

In [2]:
places = [{'county' : 'Merida',
           'state' : 'Yucatan',
           'country' : 'Mexico'}]

def get_roads_osmnx(places, update=False, proj=False, crs=None):

    dirpath = pathlib.Path('./data/networks/')
    filepath = dirpath/'merida-road.graphml'
    logpath = dirpath/'log'
                                    
    if filepath.exists() and not update:
        G = ox.load_graphml(filepath)
    else:
        # get drivable public streets network, aka road network, without service roads,
        # e.g. private, parking lots, etc.
        # use retain_all if you want to keep all disconnected subgraphs (e.g. when your places aren't adjacent)
        # TODO: It would be nice to setup up a polygon for the city and its surrounding areas, to be sure
        # exactly the location.
        G = ox.graph_from_place(places, network_type='drive')
        ox.save_graphml(G, filepath=filepath, gephi=False)
        
    if proj:
        G = ox.project_graph(G, to_crs=crs)
    
    print(f"Graph created at: {G.graph['created_date']}")
    return G, *ox.graph_to_gdfs(G)
        
#G, nodes, edges = get_roads_osmnx(places, update=False)
G_proj, nodes_proj, edges_proj = get_roads_osmnx(places, update=False, proj=True, crs=3857)

Graph created at: 2021-03-31 01:02:44


In [3]:
# get largest strongly connected component for calculating shortest paths
Gs = ox.utils_graph.get_largest_component(G_proj, strongly=True)

## Construct the coarse-grained distance matrix

The distance matrix stores the minimum shortest path length between AGEBs.

The matrix is of size (NUMBER OF AGEBS x NUMBER OF AGEBS)

In [5]:
agebs = gpd.read_file('./data/merida_ageb_stats_census_shp/merida_ageb_stats_census.shp')

In [6]:
num_agebs = len(agebs)

In [7]:
distance_matrix = np.zeros((num_agebs, num_agebs))

### Multiprocess (parallel) implementation

This parallel implementation was adapted from: https://stackoverflow.com/questions/13993674/using-multiprocessing-for-finding-network-paths

In [23]:
def worker(inqueue, output):
    shortest_paths = []
    #count = 0
    for pair in iter(inqueue.get, sentinel):
        source, target = pair
        try:
            sp = nx.shortest_path_length(Gs, source=source, target=target, weight='length', method='dijkstra')
            shortest_paths.append(sp)
        except Exception as e:
            #print('Source AGEB {} to target AGEB {} failed: {}'.format(source_ageb, target_ageb, e))
            pass
            #count += 1
            #if count % 10 == 0:
            #    logger.info('{c}'.format(c = count))
    output.put(shortest_paths)

In [24]:
def test_workers(sources, targets):
    result = []
    inqueue = mp.Queue()
    for source, target in itertools.product(sources, targets):
        inqueue.put((source, target))
    procs = [mp.Process(target = worker, args = (inqueue, output))
             for i in range(mp.cpu_count())]
    for proc in procs:
        proc.daemon = True
        proc.start()
    for proc in procs:    
        inqueue.put(sentinel)
    for proc in procs:
        result.extend(output.get())
    for proc in procs:
        proc.join()
    return result

In [ ]:
%%time

sentinel = None
output = mp.Queue()

for source_ageb in range(num_agebs):
    for target_ageb in range(num_agebs):
        source_nodes = ageb_centroids['nodes'].iloc[source_ageb]
        target_nodes = ageb_centroids['nodes'].iloc[target_ageb]
        
        if (source_ageb == target_ageb):
            min_sp = 0.
        else:
            shortest_paths = test_workers(source_nodes, target_nodes)
            min_sp = min(shortest_paths, default = 0.)
            
        distance_matrix[source_ageb][target_ageb] = min_sp
        np.save('dist_matrix', distance_matrix) # save matrix in binary file
        print("Source AGEB: " + str(source_ageb))
        print("Target AGEB: " + str(target_ageb))
        print("Minimum shortest path length: " + str(min_sp))

Source AGEB: 0
Target AGEB: 0
0.0
Source AGEB: 0
Target AGEB: 1
2927.134
Source AGEB: 0
Target AGEB: 2
2062.024
Source AGEB: 0
Target AGEB: 3
853.208


Process Process-14:
Process Process-16:
Process Process-13:
Process Process-15:
Traceback (most recent call last):


In [ ]:
distance_matrix

In [2]:
dist_mat = np.load('dist_matrix.npy') # load the saved matrix

In [ ]:
dist_mat == distance_matrix # check if they are the same